BÁO CÁO BÀI TẬP CUỐI KỲ
FACE RECOGNITION SỬ DỤNG KNN CLASSIFIER 

HV: Bùi Văn Hồ
MSHV: CH1702034

I. Mô tả bài toán
                  Process Face recognition

1. Chọn và sử dụng một hình ảnh bất kỳ (chứa một hoặc nhiều người với nhiều khuôn mặt khác nhau) làm ảnh đầu vào

2. Sử dụng API được cung cấp sẵn từ module face_recognition để xác định vị trí của từng khuôn mặt trong ảnh đầu vào

3. Sử dụng model KNN để dự đoán thông tin từng khuôn mặt (cụ thể là tên của từng người), sau đó thông tin dự đoán được sẽ được gán cho từng khuôn mặt tương ứng.

4. Kết quả của bài toán là xuất ra hình ảnh ban đầu, định vị khuôn mặt và dán nhãn (tên) cho từng khuôn mặt có trên hình ảnh ban đầu.

II. CÁC BƯỚC THỰC HIỆN

1. Download file facetest.py, test.zip, train.zip theo đường dẫn: https://github.com/vanhosgit2010/BT-CUOI-KY
2. Công cụ sữ dụng là JetBrains PyCharm 2018.3.2 x64 
3. Tổ chức thư mục lưu trử

        <train>/
        ├── <person1>/
        │   ├── <name1>.jpeg
        │   ├── <name2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...

        <test>/
          ├── <h1>.jpeg
          ├── <h2>.jpeg
          ├── <h3>.png
          ├── ...
      
         facetest.py
4. Chạy file facetest.py nếu xãy ra lỗi chủ yếu là lỗi chưa có thư viện, sẽ tiến hành import thư viện vào PyCharm 
5. Chờ và nhận kết quả thuật toán nhận dạng. đánh giá độ chính xác của bài toán

III.CODE 

       



In [0]:
1. # IMPORT THƯ VIỆN HỔ TRỢ
import cv2, numpy, math
from sklearn import neighbors
import os
import os.path
import matplotlib.pyplot as plt
import pickle
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder


In [0]:
2. # ĐỊNH NGHĨA KIỂU ẢNH ĐẦU VÀO
ALLOWED_EXTENSIONS = {'png', 'jpg'}

3. # HÀM TRAINING
def train(train_dir, model_save_path, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    
    # TẠO 2 MẢNG X VA Y
    X = []
    y = []

    # VÒNG LẬP DUYỆT QUA CÁC MỤC
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        # DUYỆT HÌNH TRONG CÁC THƯ MỤC CỦA TỪNG NGƯỜI
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
          
            # GỌI API
            image = face_recognition.load_image_file(img_path) 
            
            
            face_bounding_boxes = face_recognition.face_locations(image, number_of_times_to_upsample=1,model='hog')
            
            # KHÔNG THẤY (len() bằng 0) THẤY (len() > 1). 
            # => ẢNH CHỈ CHỨA KHUÔN MẶT
            if len(face_bounding_boxes) != 1:          
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # GT ENCODE VÀO X[], TÊN VÀO Y[]
                # LÁY PHẦN TỬ ĐẦU TIÊN TRONG MẢNG
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    # BAO NHIÊU neighbors XÁC ĐỊNH KNN classifier, 8
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # TẠO VÀ TRAIN
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # LƯU
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    # KQ KNN
    return knn_clf

In [0]:
def predict(X_img_path, model_path, distance_threshold=0.6):
    
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    # LOAD MODEL
    knn_clf = pickle.load(open(model_path, 'rb'))

    # LOAD ẢNH VÀ XÁC ĐỊNH TỌA ĐỘ
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img,number_of_times_to_upsample=1,model='hog')

    # KHÔNG TÌM THẤY TRẢ VỀ 0
    if len(X_face_locations) == 0:
        return []

    # ENCODING VÙNG ẢNH
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # KNN, SO SÁNH VÀ TÌM KẾT QUẢ
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # KQ TRẢ VỀ TƯƠNG ỨNG . KHÔNG XÁC ĐỊNH THÌ 'unknown'
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]

In [0]:
# HIỂN THỊ KẾT QUẢ KHUÔN MẶT
def show_prediction_labels_on_image(img_path, predictions):
    
    # LOAD HÌNH 
    img = cv2.imread(img_path)
         

    # GIÁ TRỊ, KHUÔN MẶT VÀ TÊN 
    for name, (top, right, bottom, left) in predictions:
        
        # VẼ KHUNG 
        # cv2.rectangle(HÌNH, TÊN, TỌA ĐỘ (left,top), TỌA ĐỘ(right, bottom), MÀU, ĐỘ DẦY) 
        cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0),2)
        
        # TEXT TRÊN ẢNH 
        #  cv2.putText(HÌNH, TÊN, TỌA ĐỘ (left,top), FONT, ....)
        cv2.putText(img, name, (left, top), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

    plt.show()

In [0]:
# THÔNG BÁO TRAINING THÀNH CÔNG
if __name__ == "__main__":
    
    print("Training KNN classifier...")
    classifier = train("FaceDB", model_save_path="trained_knn_model_2.clf", n_neighbors=2)
    print("Training complete!")

    

In [0]:
    for image_file in os.listdir("knn_examples/test"):
        full_file_path = os.path.join("knn_examples/test", image_file)

        print("Looking for faces in {}".format(image_file))

        # TÌM CÁC KHUÔN MẶT ĐÃ TRAIN TRƯỚC ĐÓ
        # Note: You can pass in either a classifier file name or a classifier model instance
        predictions = predict(full_file_path, model_path="trained_knn_model.clf")

        # IN RA KẾT QUẢ
        for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))

        # XUẤT RA MÀN HÌNH 

        show_prediction_labels_on_image(os.path.join("knn_examples/test", image_file), predictions)
        

Chạy file và xem kết quả: https://github.com/vanhosgit2010/BT-CUOI-KY/blob/master/K%E1%BA%BFt%20qu%E1%BA%A3%20sau%20khi%20training.docx



IV. Đánh giá

Số lương thư mục tương ứng mỗi người: 17 thư mục
Tổng số ảnh: 42 ảnh (trung bình một thư mục có ít nhất 1 ảnh )
Số lương ảnh test: 7 ảnh

  Hình 1
Detect và predict: 2/2
Nhận xét: Ảnh được chụp với góc tốt

  Hình 2
Tổng số khuôn mặt: 18
Detect và predict: 12/19
Nhận xét: Khuôn mặt nhỏ do chụp xa, tỉ lệ detect không cao


  Hình 3
 Tổng số khuôn mặt: 3
Detect và predict: 2/3
Nhận xét: Ảnh được chụp với góc tốt

  Hình 4
Tổng số khuôn mặt: 15
Detect và predict: 7/15
Nhận xét: Khuôn mặt nhỏ do chụp xa, tỉ lệ detect không cao

  Hình 5
Tổng số khuôn mặt: 11
Detect và predict: 8/11
Nhận xét: trực diện, ảnh có góc khuất

  Hình 6
Tổng số khuôn mặt: 2
Detect và predict: 2/2
Nhận xét: good

  Hình 7
Detect và predict: 2/2
Nhận xét: Ảnh được chụp với góc tốt


V. Kết luận
Kết quả cho thấy, tỉ lệ thành công trong bài toán nhận diện khuôn mặt phụ thuộc rất nhiều điều: Ảnh đầu vào (input):  với nhiều góc độ, độ sáng khác nhau, chất lương hình ảnh ...Dataset và Thuật toán: dataset trong việc training cần cần lượng lớn hình ảnh khác nhau